## Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

### Introduction

In order to increase market share of starbucks coffee shop and fullfill coffee demand in Vancouver, Starbucks want to set up a new store . We need to compare the neighborhoods of Vancouver and determine how similar or dissimilar they are. As a result, we would need to give a recommedation of new store location.

### Dataset

There are two dataset will be used in analynize:

1. directory.csv : Contains the info of starbucks stores in Vancouver, including address,store num,etc..
2. neighborhoods data from City of Vancouver: Contains all neighborhoods in Vancouver
3. geopy library to find location data for all neighborhoods
4. Foursquare library to find venues in each neighborhood, which will be used in clustering



### Methodology and Analysis

I will find venue types and counts in each neighborhoods and compare the number of starbucks stores in each areas. 

In [437]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [438]:
from bs4 import BeautifulSoup
url = 'https://vancouver.ca/news-calendar/areas-of-the-city.aspx'

html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

neighborhood_list = soup.find("div", {"role": "navigation"}).find_all('ul')[1].find_all('a')
neighborhood_list

[<a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1"><div></div></a>,
 <a class="menulink" href="/news-calendar/arbutus-ridge.aspx?">Arbutus Ridge</a>,
 <a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1"><div></div></a>,
 <a class="menulink" href="/news-calendar/downtown.aspx?">Downtown</a>,
 <a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1"><div></div></a>,
 <a class="menulink" href="/news-calendar/dunbar-southlands.aspx?">Dunbar-Southlands</a>,
 <a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1"><div></div></a>,
 <a class="menulink" href="/news-calendar/fairview.aspx?">Fairview</a>,
 <a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1"><div></div></a>,
 <a class="menulink" href="/news-calendar/grandview-woodland.aspx?">Grandview-Woodland   </a>,
 <a href="#" onclick="window.location = $(this).next().attr('href');" tabindex="-1">

In [439]:
# Create Pandas dataframe

column_names = ['Neighborhood']

df = pd.DataFrame(columns = column_names)

for i in neighborhood_list:
    if (i.text != ''):
        df = df.append({'Neighborhood': i.text}, ignore_index=True)

df

,Neighborhood
0,Arbutus Ridge
1,Downtown
2,Dunbar-Southlands
3,Fairview
4,Grandview-Woodland
5,Hastings-Sunrise
6,Kensington-Cedar Cottage
7,Kerrisdale
8,Killarney
9,Kitsilano


In [440]:
geolocator = Nominatim(user_agent="foursquare_agent")

for index, row in df.iterrows():
    neighborhood = row['Neighborhood']

    location = geolocator.geocode('{}, Vancouver, British Columbia'.format(neighborhood))
    latitude = location.latitude
    longitude = location.longitude
    
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude
    
df

,Neighborhood,Latitude,Longitude
0,Arbutus Ridge,49.240968,-123.167001
1,Downtown,49.283393,-123.117456
2,Dunbar-Southlands,49.253460,-123.185044
3,Fairview,49.264113,-123.126835
4,Grandview-Woodland,49.270559,-123.067942
5,Hastings-Sunrise,49.277594,-123.043920
6,Kensington-Cedar Cottage,49.247632,-123.084207
7,Kerrisdale,49.234673,-123.155389
8,Killarney,49.224274,-123.046250
9,Kitsilano,49.269410,-123.155267


In [441]:
print('The dataframe has {} neighborhoods.'.format(
        len(df['Neighborhood'].unique()),
        df.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [442]:
address = 'Vancouver, BC'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [443]:
map_van = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_van)  
    
map_van

In [444]:
CLIENT_ID = 'Q3UNBDY4QKRXKLFVM11BBGKZIBALBV4CJ10S3H1F5NFMOBJJ' # your Foursquare ID
CLIENT_SECRET = 'JCN2HEYI1JAXUE4ZT4XSSEHGRF1QS134GCZMCCTFRAFGZ31D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q3UNBDY4QKRXKLFVM11BBGKZIBALBV4CJ10S3H1F5NFMOBJJ
CLIENT_SECRET:JCN2HEYI1JAXUE4ZT4XSSEHGRF1QS134GCZMCCTFRAFGZ31D


In [445]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500

In [446]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [447]:
van_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Arbutus Ridge
Downtown
Dunbar-Southlands
Fairview
Grandview-Woodland   
Hastings-Sunrise
Kensington-Cedar Cottage
Kerrisdale
Killarney
Kitsilano
Marpole
Mount Pleasant
Oakridge
Renfrew-Collingwood
Riley Park
Shaughnessy
South Cambie  
Strathcona
Sunset
Victoria-Fraserview
West End
West Point Grey


In [448]:
print(van_venues.shape)
van_venues.head()

(671, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbutus Ridge,49.240968,-123.167001,Butter Baked Goods,49.242209,-123.170381,Bakery
1,Arbutus Ridge,49.240968,-123.167001,The Haven,49.241377,-123.166331,Spa
2,Arbutus Ridge,49.240968,-123.167001,Barktholomews Pet Supplies,49.242746,-123.170193,Pet Store
3,Arbutus Ridge,49.240968,-123.167001,The Dragon's Layer,49.238518,-123.169029,Nightlife Spot
4,Arbutus Ridge,49.240968,-123.167001,The Heights Market,49.237902,-123.170949,Grocery Store


In [449]:
print('There are {} uniques categories.'.format(len(van_venues['Venue Category'].unique())))
                                                    

There are 162 uniques categories.


In [450]:
van_grouped = van_onehot.groupby('Neighborhood').mean().reset_index()
van_grouped_coffee  = van_grouped[van_grouped.columns[van_grouped.columns.str.contains('Coffee Shop')]]
van_grouped_coffee['Neighborhood'] = van_grouped['Neighborhood']
van_grouped_coffee

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Coffee Shop,Neighborhood
0,0.000000,Arbutus Ridge
1,0.050000,Downtown
2,0.142857,Dunbar-Southlands
3,0.148148,Fairview
4,0.098592,Grandview-Woodland
5,0.071429,Hastings-Sunrise
6,0.125000,Kensington-Cedar Cottage
7,0.102564,Kerrisdale
8,0.000000,Killarney
9,0.041667,Kitsilano


In [451]:
van_grouped_coffee_top = van_grouped_coffee.sort_values(by=['Coffee Shop'],ascending=False)
van_grouped_coffee_top.head(5)

,Coffee Shop,Neighborhood
16,0.250000,South Cambie
3,0.148148,Fairview
2,0.142857,Dunbar-Southlands
6,0.125000,Kensington-Cedar Cottage
17,0.125000,Strathcona


In [452]:
num_top_venues = 5

for hood in van_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = van_grouped[van_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arbutus Ridge----
            venue  freq
0             Spa   0.2
1  Nightlife Spot   0.2
2   Grocery Store   0.2
3          Bakery   0.2
4       Pet Store   0.2


----Downtown----
                   venue  freq
0                  Hotel  0.09
1            Coffee Shop  0.05
2                   Café  0.04
3             Restaurant  0.03
4  Vietnamese Restaurant  0.03


----Dunbar-Southlands----
                 venue  freq
0  Sporting Goods Shop  0.14
1  Japanese Restaurant  0.14
2   Italian Restaurant  0.14
3     Sushi Restaurant  0.14
4    Indian Restaurant  0.14


----Fairview----
               venue  freq
0        Coffee Shop  0.15
1   Asian Restaurant  0.07
2               Park  0.07
3              Diner  0.04
4  Indian Restaurant  0.04


----Grandview-Woodland   ----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2    Sushi Restaurant  0.04
3         Pizza Place  0.04
4   Indian Restaurant  0.04


----Hastings-Sunrise----
                   

In [453]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [466]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = van_grouped['Neighborhood']

for ind in np.arange(van_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(van_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arbutus Ridge,Spa,Grocery Store,Pet Store,Nightlife Spot,Bakery,Food Truck,Food Court,Food,Flower Shop,Filipino Restaurant
1,Downtown,Hotel,Coffee Shop,Café,Concert Hall,Restaurant,Dessert Shop,Sandwich Place,Clothing Store,Steakhouse,Seafood Restaurant
2,Dunbar-Southlands,Japanese Restaurant,Sushi Restaurant,Indian Restaurant,Ice Cream Shop,Coffee Shop,Sporting Goods Shop,Italian Restaurant,BBQ Joint,Furniture / Home Store,Fried Chicken Joint
3,Fairview,Coffee Shop,Park,Asian Restaurant,Korean Restaurant,Sandwich Place,Salon / Barbershop,Malay Restaurant,Restaurant,Falafel Restaurant,Chinese Restaurant
4,Grandview-Woodland,Coffee Shop,Italian Restaurant,Japanese Restaurant,Café,Sushi Restaurant,Park,Indian Restaurant,Pizza Place,French Restaurant,Record Shop
5,Hastings-Sunrise,Vietnamese Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Sushi Restaurant,Event Space,Food Truck,Bakery,Park,Gas Station
6,Kensington-Cedar Cottage,Coffee Shop,Bus Stop,Filipino Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pizza Place,Sandwich Place,Malay Restaurant,Restaurant,Gourmet Shop
7,Kerrisdale,Coffee Shop,Chinese Restaurant,Tea Room,Pharmacy,Sushi Restaurant,Sandwich Place,Pizza Place,Business Service,Café,Portuguese Restaurant
8,Killarney,Italian Restaurant,Track,Gym,Pool,Ice Cream Shop,French Restaurant,Food Court,Food,Flower Shop,Filipino Restaurant
9,Kitsilano,Bakery,Japanese Restaurant,Thai Restaurant,Coffee Shop,Food Truck,French Restaurant,Ice Cream Shop,Sushi Restaurant,American Restaurant,Indian Restaurant


In [458]:
df_starbucks = pd.read_csv('directory.csv')

In [459]:
df_starbucks= df_starbucks[df_starbucks['Country'] == "CA"]
df_starbucks = df_starbucks[df_starbucks['State/Province'] == "BC"]
df_starbucks = df_starbucks[df_starbucks['City'] == "Vancouver"]
df_starbucks.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
1006,Starbucks,4359-1858,Pacific Blvd & Davie/Concord,Company Owned,125 Davie Street,Vancouver,BC,CA,V6Z 2Y1,604-609-7717,GMT-08:00 America/Vancouver,-123.12,49.27
1007,Starbucks,75973-92562,Westin Bayshore,Licensed,1601 Bayshore Dr,Vancouver,BC,CA,V6G 2V4,NaN,GMT-08:00 America/Vancouver,-123.13,49.29
1008,Starbucks,4576-96682,Cornwall & Cypress,Company Owned,"1895 Cornwall Ave, Vancouver Centre",Vancouver,BC,CA,V6J 1C6,604-733-6211,GMT-08:00 America/Vancouver,-123.15,49.27
1009,Starbucks,4410-30299,Granville & Georgia,Company Owned,720 Granville St,Vancouver,BC,CA,V6Z 1E4,604/633-9801,GMT-08:00 America/Vancouver,-123.12,49.28
1010,Starbucks,4454-82765,1855 Burrard Street,Company Owned,1855 Burrard Street,Vancouver,BC,CA,V6J 3G9,604-733-4205,GMT-08:00 America/Vancouver,-123.15,49.27


In [460]:
df_starbucks.shape

(110, 13)

In [461]:
nei_list = []
coordinate_list = []

for index, row in van_merged.iterrows():
    
    coordinate = (row['Latitude'], row['Longitude'])
    
    nei_list.append(row['Neighborhood'])
    coordinate_list.append(coordinate)
    
nei_locations = [nei_list, coordinate_list]
nei_locations

[['Arbutus Ridge',
  'Downtown',
  'Dunbar-Southlands',
  'Fairview',
  'Grandview-Woodland   ',
  'Hastings-Sunrise',
  'Kensington-Cedar Cottage',
  'Kerrisdale',
  'Killarney',
  'Kitsilano',
  'Marpole',
  'Mount Pleasant',
  'Oakridge',
  'Renfrew-Collingwood',
  'Riley Park',
  'Shaughnessy',
  'South Cambie  ',
  'Strathcona',
  'Sunset',
  'Victoria-Fraserview',
  'West End',
  'West Point Grey'],
 [(49.2409677, -123.1670008),
  (49.283393, -123.1174563),
  (49.2534601, -123.1850439),
  (49.2641128, -123.1268352),
  (49.2705588, -123.0679417),
  (49.2775935, -123.0439199),
  (49.2476321, -123.0842067),
  (49.2346728, -123.1553893),
  (49.2242738, -123.0462504),
  (49.2694099, -123.155267),
  (49.2092233, -123.1361495),
  (49.2633296, -123.0965885),
  (49.2308288, -123.1311342),
  (49.2420242, -123.0576794),
  (49.2474381, -123.1029664),
  (49.2518626, -123.1380226),
  (49.2466847, -123.120915),
  (49.279554, -123.0899788),
  (49.2195929, -123.0902386),
  (49.2184156, -123.07328

In [462]:
import geopy.distance

def find_closest_neighborhood(coord, nei_locations):
    index = 0
    min_dist = 999999999999999999999
    
    for i in range(len(nei_locations[1])):
        coords_2 = nei_locations[1][i]
        dist = geopy.distance.distance(coord, coords_2).km
        
        if dist < min_dist:
            min_dist = dist
            index = i
            
    return nei_locations[0][index]

In [463]:
nei_locations = nei_locations

for index, row in df_starbucks.iterrows():
    
    starbucks_to_find = (row['Latitude'], row['Longitude'])
    
    df_starbucks.at[index, 'Neighborhood'] = find_closest_neighborhood(starbucks_to_find, nei_locations)
    
df_starbucks.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Neighborhood
1006,Starbucks,4359-1858,Pacific Blvd & Davie/Concord,Company Owned,125 Davie Street,Vancouver,BC,CA,V6Z 2Y1,604-609-7717,GMT-08:00 America/Vancouver,-123.12,49.27,Fairview
1007,Starbucks,75973-92562,Westin Bayshore,Licensed,1601 Bayshore Dr,Vancouver,BC,CA,V6G 2V4,NaN,GMT-08:00 America/Vancouver,-123.13,49.29,West End
1008,Starbucks,4576-96682,Cornwall & Cypress,Company Owned,"1895 Cornwall Ave, Vancouver Centre",Vancouver,BC,CA,V6J 1C6,604-733-6211,GMT-08:00 America/Vancouver,-123.15,49.27,Kitsilano
1009,Starbucks,4410-30299,Granville & Georgia,Company Owned,720 Granville St,Vancouver,BC,CA,V6Z 1E4,604/633-9801,GMT-08:00 America/Vancouver,-123.12,49.28,Downtown
1010,Starbucks,4454-82765,1855 Burrard Street,Company Owned,1855 Burrard Street,Vancouver,BC,CA,V6J 3G9,604-733-4205,GMT-08:00 America/Vancouver,-123.15,49.27,Kitsilano


In [476]:
df_starbucks_num = df_starbucks.groupby('Neighborhood')['Store Number'].count().to_frame()
df_starbucks_num

,Store Number
Neighborhood,
Downtown,31
Dunbar-Southlands,1
Fairview,6
Grandview-Woodland,4
Hastings-Sunrise,5
Kensington-Cedar Cottage,1
Kerrisdale,3
Killarney,3
Kitsilano,8


In [465]:
van_grouped_coffee

,Coffee Shop,Neighborhood
0,0.000000,Arbutus Ridge
1,0.050000,Downtown
2,0.142857,Dunbar-Southlands
3,0.148148,Fairview
4,0.098592,Grandview-Woodland
5,0.071429,Hastings-Sunrise
6,0.125000,Kensington-Cedar Cottage
7,0.102564,Kerrisdale
8,0.000000,Killarney
9,0.041667,Kitsilano


In [486]:
# How many venues each neighborhood has

venue_count = van_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
venue_count.rename(columns={'Venue': 'Venue Count'}, inplace=True)
venue_count

,Venue Count
Neighborhood,
Arbutus Ridge,5
Downtown,100
Dunbar-Southlands,7
Fairview,27
Grandview-Woodland,71
Hastings-Sunrise,14
Kensington-Cedar Cottage,24
Kerrisdale,39
Killarney,4


In [492]:
col_name="Neighborhood"
first_col = van_grouped_coffee.pop(col_name)
van_grouped_coffee.insert(0, col_name, first_col)
df_merge = pd.merge(left = van_grouped_coffee ,right = df_starbucks_num ,on = 'Neighborhood')
df_merge_new = pd.merge(left = venue_count, right = df_merge,on = 'Neighborhood' )
df_merge_new.rename(columns={'Store Number': 'Number of Starbucks'}, inplace=True)
df_merge_new

,Neighborhood,Venue Count,Coffee Shop,Number of Starbucks
0,Downtown,100,0.050000,31
1,Dunbar-Southlands,7,0.142857,1
2,Fairview,27,0.148148,6
3,Grandview-Woodland,71,0.098592,4
4,Hastings-Sunrise,14,0.071429,5
5,Kensington-Cedar Cottage,24,0.125000,1
6,Kerrisdale,39,0.102564,3
7,Killarney,4,0.000000,3
8,Kitsilano,48,0.041667,8
9,Marpole,31,0.000000,4


### Result and Discussion

In [499]:
df_merge_new

,Neighborhood,Venue Count,Coffee Shop,Number of Starbucks
0,Downtown,100,0.050000,31
1,Dunbar-Southlands,7,0.142857,1
2,Fairview,27,0.148148,6
3,Grandview-Woodland,71,0.098592,4
4,Hastings-Sunrise,14,0.071429,5
5,Kensington-Cedar Cottage,24,0.125000,1
6,Kerrisdale,39,0.102564,3
7,Killarney,4,0.000000,3
8,Kitsilano,48,0.041667,8
9,Marpole,31,0.000000,4


As can be see from the above table, Riley Park has 58 venues, but the proportion of coffee shop is only 5.17%. Also, there is only one Starbucks, so we need open more starbucks around this area to fullfill the coffee shop market. Compare Riley Park to Kitsilano, we could see Kitsilano has much more Starbucks; however, Kitsilano has lower proportion of coffee shop than Riley Park. 

#### Let's see top 10 venues in Riley Park and Kitsilano

In [496]:
neighborhoods_venues_sorted.query('Neighborhood== "Riley Park"')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Riley Park,Japanese Restaurant,Café,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Pub,Chinese Restaurant,Thai Restaurant,Lounge,Arts & Crafts Store


In [498]:
neighborhoods_venues_sorted.query('Neighborhood== "Kitsilano"')

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Kitsilano,Bakery,Japanese Restaurant,Thai Restaurant,Coffee Shop,Food Truck,French Restaurant,Ice Cream Shop,Sushi Restaurant,American Restaurant,Indian Restaurant


We could see Riley Park has similar venue types with Kitsilano. They both have a lot international restaurant, but Kitsilano has much more starbucks. Thus, we need to open more Starbucks in Riley Park.

### Conclusion

Based on analysis, I recommend locate the new Starbucks Store in Riley Park Area. 